In [3]:
!pip install dataloader

  Preparing metadata (setup.py) ... done
  Created wheel for dataloader: filename=dataloader-2.0-py3-none-any.whl size=10084 sha256=ad2415ec9981386d72824c0b5e3c2c4cabad7a1668050bb496cc5853db095e75
  Stored in directory: /Users/lennoxanderson/Library/Caches/pip/wheels/fa/17/c3/e258aa863dd515a9b5759c01f4af55b2a2ea0881b2778fc749
Successfully built dataloader


In [33]:
import pandas as pd
import os
from pathlib import Path
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class DataLoader:
    def __init__(self, data_dir='../ETL/data/'):
        """Initialize DataLoader with the data directory path."""
        self.data_dir = Path(data_dir)
        if not self.data_dir.exists():
            raise FileNotFoundError(f"Directory {data_dir} does not exist")
        
    def list_available_files(self):
        """List all CSV and Excel files in the data directory."""
        files = []
        for ext in ['*.csv', '*.xlsx']:
            files.extend(list(self.data_dir.glob(ext)))
        return files
    
    def load_survey_results(self):
        """Load survey results data."""
        try:
            survey_files = list(self.data_dir.glob('survey_results*.csv'))
            if not survey_files:
                logger.warning("No survey results files found")
                return None
            
            latest_file = max(survey_files)
            logger.info(f"Loading survey data from {latest_file}")
            return pd.read_csv(latest_file)
            
        except Exception as e:
            logger.error(f"Error loading survey results: {e}")
            raise
    
    def load_users_data(self):
        """Load users data."""
        try:
            users_files = list(self.data_dir.glob('users_data*.csv'))
            if not users_files:
                logger.warning("No users data files found")
                return None
            
            latest_file = max(users_files)
            logger.info(f"Loading users data from {latest_file}")
            return pd.read_csv(latest_file)
            
        except Exception as e:
            logger.error(f"Error loading users data: {e}")
            raise

In [35]:
# Initialize loader
loader = DataLoader()

# Load all available data
survey_data = loader.load_survey_results()
users_data = loader.load_users_data()

# Check what files are available
print("Available files:")
for file in loader.list_available_files():
    print(f"- {file.name}")

2025-01-28 18:09:29,640 - INFO - Loading survey data from ../ETL/data/survey_results_20250128_180837.csv
2025-01-28 18:09:29,680 - INFO - Loading users data from ../ETL/data/users_data_20250128_180923.csv


Available files:
- users_data_20250128_180923.csv
- survey_results_20250128_180837.csv


In [37]:
# Initialize loader and load data into DataFrames
loader = DataLoader()

# Load survey results into DataFrame
survey_df = loader.load_survey_results()
print("\nSurvey Results DataFrame Info:")
survey_df.info()
print("\nFirst few rows of survey data:")
survey_df.head()

# Load users data into DataFrame
users_df = loader.load_users_data()
print("\nUsers Data DataFrame Info:")
users_df.info()
print("\nFirst few rows of users data:")
users_df.head()

# Basic information about the DataFrames
print("\nSurvey DataFrame Shape:", survey_df.shape if survey_df is not None else "No data")
print("Users DataFrame Shape:", users_df.shape if users_df is not None else "No data")

2025-01-28 18:09:30,266 - INFO - Loading survey data from ../ETL/data/survey_results_20250128_180837.csv
2025-01-28 18:09:30,307 - INFO - Loading users data from ../ETL/data/users_data_20250128_180923.csv



Survey Results DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               311 non-null    object
 1   userId            311 non-null    object
 2   videoId           311 non-null    object
 3   windowDimensions  311 non-null    object
 4   gaze              311 non-null    object
 5   formData          311 non-null    object
 6   __v               311 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 17.1+ KB

First few rows of survey data:

Users Data DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               21 non-null     object
 1   email             21 non-null     object
 2   password          21 non-null     object
 3   re

In [39]:
users_df

,_id,email,password,referralCode,referredByUser,numSurveysFilled,numRaffleEntries,form,__v
0,6795646390793f4f61cbbbcc,jonahmulcrone@gmail.com,$2b$10$5m8KlPib6gp8rLt2dyWl.O1PHy4hSzMP0yFd6uM...,8f2ae8ed-e460-46c6-ae38-e6d47089cbfb,NaN,6,11,"{'country': 'US', 'state': 'Washington', 'city...",0
1,6795679190793f4f61cbbc1c,lisamulcrone02@gmail.com,$2b$10$WjzM5xcHrl08GEOsLMxk6OgmiZe5b/BE9Ll4ddb...,c39c3a45-5229-4fdc-a932-0f37a7b87bb7,NaN,10,15,"{'country': 'US', 'state': 'Washington', 'city...",0
2,6797a06590793f4f61cbbe62,jhoffman@flexgen.com,$2b$10$Ni.B/3dpVeSryK5O7N7j6.zVTa.D/sbaS/Y.ajC...,6ddc1dd7-5c9a-4ac7-b67c-354516bfe39a,NaN,29,44,"{'country': 'US', 'state': 'North Carolina', '...",0
3,6797a4db90793f4f61cbc060,akwon@flexgen.com,$2b$10$0d6RUPGC8D5wkkr55TD/DObeQH7FQ6iPfCIeDm8...,9590b111-7e80-41a1-976f-4d8c3b7e9ed8,NaN,11,16,"{'country': 'US', 'state': 'North Carolina', '...",0
4,6797a4ea90793f4f61cbc066,bsullivan@flexgen.com,$2b$10$IwZkPSXsNhJpqbGz2adrxO4UX9Ugl2TURtz7XiY...,36811ea4-5c07-4736-be6f-ec77297a57ef,6ddc1dd7-5c9a-4ac7-b67c-354516bfe39a,13,18,"{'country': 'US', 'state': 'North Carolina', '...",0
5,6797c06590793f4f61cbc7de,andersonlennox381@outlook.com,$2b$10$cnOEb2BUmzNH8g2wt6yG3uTruts9OfdhwV1YSSO...,40b59a74-5c85-4c2b-bbd8-8c057aeb9482,NaN,5,10,"{'country': 'US', 'state': 'North Carolina', '...",0
6,6798188690793f4f61cbc7fd,andrew.rodov@gmail.com,$2b$10$sIvwy2WT/iwlgk2VqNwJyuYVLxREZ27rVSol.YF...,da435f23-a3ec-4639-9ff7-447d7a226df7,NaN,12,17,"{'country': 'US', 'state': 'Florida', 'city': ...",0
7,67981a2890793f4f61cbcc94,ryanhakim3@gmail.com,$2b$10$2XLZ6Ugx/s6hu7T4OjqDyO4pSuQ2GKtlr57R2f1...,e11ae071-216e-4aaa-94cd-60274eef12ba,NaN,14,19,"{'country': 'US', 'state': 'California', 'city...",0
8,6798401490793f4f61cbd348,goldringsigal@gmail.com,$2b$10$e/o7wAFh17hvrnxJtLtTHO0comn5eUQYBhtxezr...,28db2a7b-9618-4fcc-ba3a-e5d0744c0d87,NaN,26,31,"{'country': 'US', 'state': 'Florida', 'city': ...",0
9,6798496890793f4f61cbd982,sy1328@gmail.com,$2b$10$tVFyQGhvewprDVFwX39FTezE80Hnlm5Mvq1dJ73...,dcfec6f1-0ce8-460a-b2d1-4a77ab64463a,NaN,17,22,"{'country': 'US', 'state': 'Florida', 'city': ...",0


In [41]:
survey_df

,_id,userId,videoId,windowDimensions,gaze,formData,__v
0,679564a790793f4f61cbbbd2,jonahmulcrone@gmail.com,video219,"{'width': 1470, 'height': 797, '_id': ObjectId...","[{'x': 500.4967634374761, 'y': 499.95337818871...","{'hazardDetected': False, 'noDetectionReason':...",0
1,679568ec90793f4f61cbbc22,lisamulcrone02@gmail.com,video537,"{'width': 1440, 'height': 783, '_id': ObjectId...","[{'x': 500.22292728154264, 'y': 499.9839235133...","{'hazardDetected': True, 'noDetectionReason': ...",0
2,6795692990793f4f61cbbca7,lisamulcrone02@gmail.com,video452,"{'width': 1440, 'height': 783, '_id': ObjectId...","[{'x': 500.21810433554776, 'y': 500.1516548573...","{'hazardDetected': True, 'noDetectionReason': ...",0
3,6795695490793f4f61cbbd0e,lisamulcrone02@gmail.com,video71,"{'width': 1440, 'height': 783, '_id': ObjectId...","[{'x': 500.15754956916714, 'y': 500.0728800728...","{'hazardDetected': False, 'noDetectionReason':...",0
4,6795697790793f4f61cbbd55,lisamulcrone02@gmail.com,video188,"{'width': 1440, 'height': 783, '_id': ObjectId...","[{'x': 500.11628658676625, 'y': 500.0176841353...","{'hazardDetected': False, 'noDetectionReason':...",0
...,...,...,...,...,...,...,...
306,67994fce90793f4f61cc285f,kimgalvao96@gmail.com,video166,"{'width': 2319, 'height': 1148, '_id': ObjectI...","[{'x': 500.02572237863956, 'y': 500.0257223786...","{'hazardDetected': True, 'noDetectionReason': ...",0
307,67995fa090793f4f61cc289c,jonahmulcrone@gmail.com,video209,"{'width': 1470, 'height': 797, '_id': ObjectId...","[{'x': 500.2443625970756, 'y': 500.17898488470...","{'hazardDetected': False, 'noDetectionReason':...",0
308,67995fc090793f4f61cc28ea,jonahmulcrone@gmail.com,video71,"{'width': 1470, 'height': 797, '_id': ObjectId...","[{'x': 500.4528210406335, 'y': 500.31777855277...","{'hazardDetected': False, 'noDetectionReason':...",0
309,67995fe490793f4f61cc293e,jonahmulcrone@gmail.com,video140,"{'width': 1470, 'height': 797, '_id': ObjectId...","[{'x': 500.52623699633386, 'y': 500.1216454156...","{'hazardDetected': False, 'noDetectionReason':...",0


In [49]:
import pandas as pd
import json
from bson import ObjectId

class DataFrameTransformer:
    @staticmethod
    def flatten_dict(d, parent_key='', sep='_'):
        """Flatten nested dictionaries, handling ObjectId and other special types."""
        items = []
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            
            if isinstance(v, dict):
                items.extend(DataFrameTransformer.flatten_dict(v, new_key, sep=sep).items())
            elif isinstance(v, ObjectId):
                items.append((new_key, str(v)))
            else:
                items.append((new_key, v))
        return dict(items)

    @staticmethod
    def process_coordinates(coord_list):
        """Process list of coordinate dictionaries into separate columns."""
        if not coord_list:
            return {'x': None, 'y': None}
        # Take the first coordinate pair since that seems to be the pattern
        return {
            'x': coord_list[0].get('x'),
            'y': coord_list[0].get('y')
        }

    @staticmethod
    def transform_dataframe(df):
        """Transform the dataframe by flattening nested structures into columns."""
        # Create new dataframe with basic columns
        transformed_data = []
        
        for _, row in df.iterrows():
            flat_row = {}
            
            # Copy basic fields
            flat_row['_id'] = row['_id']
            flat_row['email'] = row['email']
            flat_row['video_id'] = row['video_id']
            
            # Process dimensions
            if isinstance(row['dimensions'], dict):
                dimensions = DataFrameTransformer.flatten_dict(row['dimensions'])
                for k, v in dimensions.items():
                    flat_row[f'dimensions_{k}'] = v
            
            # Process coordinates
            if isinstance(row['coordinates'], list):
                coords = DataFrameTransformer.process_coordinates(row['coordinates'])
                flat_row['coordinate_x'] = coords['x']
                flat_row['coordinate_y'] = coords['y']
            
            # Process hazard detection
            if isinstance(row['hazardDetection'], dict):
                hazard_data = DataFrameTransformer.flatten_dict(row['hazardDetection'])
                for k, v in hazard_data.items():
                    flat_row[f'hazard_{k}'] = v
            
            transformed_data.append(flat_row)
        
        return pd.DataFrame(transformed_data)

# Usage example
def transform_results_df(results_df):
    transformer = DataFrameTransformer()
    transformed_df = transformer.transform_dataframe(results_df)
    return transformed_df

# Example of how to use:
# transformed_df = transform_results_df(results_df)
# print(transformed_df.head())
# print("\nColumns in transformed dataframe:")
# print(transformed_df.columns.tolist())

In [57]:
import pandas as pd
import json
from pathlib import Path
import logging

class DataFrameTransformer:
    @staticmethod
    def flatten_dict(d, parent_key='', sep='_'):
        """Flatten nested dictionaries, handling special types."""
        items = []
        if isinstance(d, str):
            try:
                d = eval(d)  # Safely convert string representation to dict
            except:
                return {parent_key: d}
                
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            
            if isinstance(v, dict):
                items.extend(DataFrameTransformer.flatten_dict(v, new_key, sep=sep).items())
            else:
                items.append((new_key, v))
        return dict(items)

    @staticmethod
    def process_coordinates(coord_list):
        """Process list of coordinate dictionaries into separate columns."""
        if not coord_list or isinstance(coord_list, str):
            try:
                if isinstance(coord_list, str):
                    coord_list = eval(coord_list)
            except:
                return {'x': None, 'y': None, 'time': None}
                
        if not coord_list:
            return {'x': None, 'y': None, 'time': None}
            
        # Take the first coordinate pair
        first_coord = coord_list[0] if isinstance(coord_list, list) else coord_list
        return {
            'x': first_coord.get('x'),
            'y': first_coord.get('y'),
            'time': first_coord.get('time')
        }

    @staticmethod
    def transform_dataframe(df):
        """Transform the dataframe by flattening nested structures into columns."""
        transformed_data = []
        
        for _, row in df.iterrows():
            flat_row = {}
            
            # Copy basic fields
            flat_row['_id'] = row.get('_id')
            flat_row['email'] = row.get('userId')  # Changed to match CSV column name
            flat_row['video_id'] = row.get('videoId')  # Changed to match CSV column name
            
            # Process dimensions
            if 'windowDimensions' in row:
                dimensions = DataFrameTransformer.flatten_dict(row['windowDimensions'])
                for k, v in dimensions.items():
                    flat_row[f'dimensions_{k}'] = v
            
            # Process coordinates
            if 'gaze' in row:
                coords = DataFrameTransformer.process_coordinates(row['gaze'])
                flat_row['coordinate_x'] = coords['x']
                flat_row['coordinate_y'] = coords['y']
                flat_row['coordinate_time'] = coords['time']
            
            # Process form data
            if 'formData' in row:
                form_data = DataFrameTransformer.flatten_dict(row['formData'])
                for k, v in form_data.items():
                    flat_row[f'form_{k}'] = v
            
            transformed_data.append(flat_row)
        
        return pd.DataFrame(transformed_data)

class DataLoader:
    def __init__(self, data_dir='../ETL/data/'):
        """Initialize DataLoader with the data directory path."""
        self.data_dir = Path(data_dir)
        if not self.data_dir.exists():
            raise FileNotFoundError(f"Directory {data_dir} does not exist")
        self.transformer = DataFrameTransformer()
        
    def list_available_files(self):
        """List all CSV and Excel files in the data directory."""
        files = []
        for ext in ['*.csv', '*.xlsx']:
            files.extend(list(self.data_dir.glob(ext)))
        return files
    
    def load_survey_results(self, transform=True):
        """Load survey results data with option to transform nested structures."""
        try:
            survey_files = list(self.data_dir.glob('survey_results*.csv'))
            if not survey_files:
                logger.warning("No survey results files found")
                return None
            
            latest_file = max(survey_files)
            logger.info(f"Loading survey data from {latest_file}")
            df = pd.read_csv(latest_file)
            
            if transform:
                logger.info("Transforming nested data structures")
                df = self.transformer.transform_dataframe(df)
            
            return df
            
        except Exception as e:
            logger.error(f"Error loading survey results: {str(e)}")
            raise

In [59]:
# Initialize loader
loader = DataLoader(data_dir='../ETL/data/')  # Replace with your actual data directory path

# Load and transform the data automatically
transformed_survey_df = loader.load_survey_results(transform=True)

# Display the transformed data
print("\nTransformed Survey Results DataFrame Info:")
transformed_survey_df.info()
print("\nFirst few rows of transformed survey data:")
print(transformed_survey_df.head())

2025-01-28 18:21:33,092 - INFO - Loading survey data from ../ETL/data/survey_results_20250128_180837.csv
2025-01-28 18:21:33,125 - INFO - Transforming nested data structures



Transformed Survey Results DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   _id                       311 non-null    object 
 1   email                     311 non-null    object 
 2   video_id                  311 non-null    object 
 3   dimensions_width          311 non-null    int64  
 4   dimensions_height         311 non-null    int64  
 5   dimensions__id            311 non-null    object 
 6   coordinate_x              298 non-null    float64
 7   coordinate_y              298 non-null    float64
 8   coordinate_time           298 non-null    float64
 9   form_hazardDetected       311 non-null    bool   
 10  form_noDetectionReason    311 non-null    object 
 11  form_detectionConfidence  311 non-null    int64  
 12  form_hazardSeverity       311 non-null    int64  
 13  form_attentionFactors

In [61]:
transformed_survey_df

,_id,email,video_id,dimensions_width,dimensions_height,dimensions__id,coordinate_x,coordinate_y,coordinate_time,form_hazardDetected,form_noDetectionReason,form_detectionConfidence,form_hazardSeverity,form_attentionFactors,form_spacebarTimestamps,form_startTime,form_endTime,form__id
0,679564a790793f4f61cbbbd2,jonahmulcrone@gmail.com,video219,1470,797,679564a790793f4f61cbbbd3,500.496763,499.953378,1.737844e+12,False,noHazards,5,0,[],[],1.737844e+12,1.737844e+12,679564a790793f4f61cbbc18
1,679568ec90793f4f61cbbc22,lisamulcrone02@gmail.com,video537,1440,783,679568ec90793f4f61cbbc23,500.222927,499.983924,1.737845e+12,True,,1,1,"[motion, proximity]","[1737844880780.0, 1737844883634.0]",1.737845e+12,1.737845e+12,679568ec90793f4f61cbbca1
2,6795692990793f4f61cbbca7,lisamulcrone02@gmail.com,video452,1440,783,6795692990793f4f61cbbca8,500.218104,500.151655,1.737845e+12,True,,5,5,"[pedestrian, motion, velocity]","[1737844982473.0, 1737844984261.0]",1.737845e+12,1.737845e+12,6795692990793f4f61cbbd08
3,6795695490793f4f61cbbd0e,lisamulcrone02@gmail.com,video71,1440,783,6795695490793f4f61cbbd0f,500.157550,500.072880,1.737845e+12,False,noHazards,1,0,[],[],1.737845e+12,1.737845e+12,6795695490793f4f61cbbd4f
4,6795697790793f4f61cbbd55,lisamulcrone02@gmail.com,video188,1440,783,6795697790793f4f61cbbd56,500.116287,500.017684,1.737845e+12,False,noHazards,1,0,[],[],1.737845e+12,1.737845e+12,6795697790793f4f61cbbd79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,67994fce90793f4f61cc285f,kimgalvao96@gmail.com,video166,2319,1148,67994fce90793f4f61cc2860,500.025722,500.025722,1.738101e+12,True,,4,3,"[proximity, other]","[1738100671513.0, 1738100671949.0]",1.738101e+12,1.738101e+12,67994fce90793f4f61cc2896
307,67995fa090793f4f61cc289c,jonahmulcrone@gmail.com,video209,1470,797,67995fa090793f4f61cc289d,500.244363,500.178985,1.738105e+12,False,noHazards,4,0,[],[],1.738105e+12,1.738105e+12,67995fa090793f4f61cc28e4
308,67995fc090793f4f61cc28ea,jonahmulcrone@gmail.com,video71,1470,797,67995fc090793f4f61cc28eb,500.452821,500.317779,1.738105e+12,False,noHazards,5,0,[],[],1.738105e+12,1.738105e+12,67995fc090793f4f61cc2938
309,67995fe490793f4f61cc293e,jonahmulcrone@gmail.com,video140,1470,797,67995fe490793f4f61cc293f,500.526237,500.121645,1.738105e+12,False,uncertain,4,0,[],[],1.738105e+12,1.738105e+12,67995fe490793f4f61cc2989


In [63]:
import pandas as pd
import numpy as np

def calculate_time_differences(df):
    """Calculate time differences between start and end times for each row."""
    
    # Create a new DataFrame with only the time columns and index
    time_df = pd.DataFrame()
    
    # Extract start and end times from formData
    time_df['duration_ms'] = df['form_endTime'] - df['form_startTime']
    
    # Convert to seconds
    time_df['duration_seconds'] = time_df['duration_ms'] / 1000
    
    # Include the user and video information
    time_df['email'] = df['email']
    time_df['video_id'] = df['video_id']
    
    # Reorder columns
    time_df = time_df[['email', 'video_id', 'duration_ms', 'duration_seconds']]
    
    return time_df

# Usage example:
try:
    # Initialize loader and load data
    loader = DataLoader(data_dir='../ETL/data/')
    transformed_df = loader.load_survey_results(transform=True)
    
    # Calculate time differences
    time_analysis_df = calculate_time_differences(transformed_df)
    
    # Display basic statistics
    print("\nTime Analysis Summary:")
    print("\nBasic Statistics (in seconds):")
    print(time_analysis_df['duration_seconds'].describe())
    
    print("\nFirst few rows of time analysis:")
    print(time_analysis_df.head())
    
except Exception as e:
    print(f"Error analyzing times: {str(e)}")

2025-01-28 18:24:57,376 - INFO - Loading survey data from ../ETL/data/survey_results_20250128_180837.csv
2025-01-28 18:24:57,406 - INFO - Transforming nested data structures



Time Analysis Summary:

Basic Statistics (in seconds):
count    3.110000e+02
mean     5.588712e+06
std      9.855773e+07
min      1.103000e+01
25%      1.506600e+01
50%      1.581600e+01
75%      1.615250e+01
max      1.738085e+09
Name: duration_seconds, dtype: float64

First few rows of time analysis:
                      email  video_id  duration_ms  duration_seconds
0   jonahmulcrone@gmail.com  video219      15755.0            15.755
1  lisamulcrone02@gmail.com  video537      15007.0            15.007
2  lisamulcrone02@gmail.com  video452      15254.0            15.254
3  lisamulcrone02@gmail.com   video71      16035.0            16.035
4  lisamulcrone02@gmail.com  video188      17036.0            17.036


In [65]:
time_df

NameError: name 'time_df' is not defined

In [73]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set pandas to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

class DataFrameTransformer:
    @staticmethod
    def flatten_dict(d, parent_key='', sep='_'):
        items = []
        if isinstance(d, str):
            try:
                d = eval(d)
            except:
                return {parent_key: d}
                
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict):
                items.extend(DataFrameTransformer.flatten_dict(v, new_key, sep=sep).items())
            else:
                items.append((new_key, v))
        return dict(items)

    @staticmethod
    def transform_dataframe(df):
        transformed_data = []
        for _, row in df.iterrows():
            flat_row = {}
            flat_row['_id'] = row.get('_id')
            flat_row['email'] = row.get('userId')
            flat_row['video_id'] = row.get('videoId')
            
            # Process form data
            if 'formData' in row:
                try:
                    form_data = DataFrameTransformer.flatten_dict(row['formData'])
                    for k, v in form_data.items():
                        flat_row[f'form_{k}'] = v
                except Exception as e:
                    logger.error(f"Error processing form data for row {row.get('_id')}: {str(e)}")
            
            transformed_data.append(flat_row)
        return pd.DataFrame(transformed_data)

def analyze_time_differences(csv_path):
    try:
        # Read the CSV file
        df = pd.read_csv(csv_path)
        logger.info(f"Successfully read CSV file with {len(df)} rows")

        # Transform the data
        transformer = DataFrameTransformer()
        transformed_df = transformer.transform_dataframe(df)
        logger.info("Successfully transformed data")

        # Calculate duration
        transformed_df['duration_ms'] = transformed_df['form_endTime'] - transformed_df['form_startTime']
        transformed_df['duration_seconds'] = transformed_df['duration_ms'] / 1000

        # Create a new DataFrame with just the timing information
        time_analysis_df = transformed_df[[
            'email', 
            'video_id', 
            'form_startTime', 
            'form_endTime', 
            'duration_ms', 
            'duration_seconds'
        ]]

        # Print full DataFrame
        print("\nComplete Time Analysis Results:")
        print(time_analysis_df.to_string())

        print("\nTime Analysis Summary:")
        print("\nBasic Statistics (in seconds):")
        print(time_analysis_df['duration_seconds'].describe())

        return time_analysis_df

    except Exception as e:
        logger.error(f"Error in time analysis: {str(e)}")
        raise

if __name__ == "__main__":
    # Replace with your actual CSV path
    csv_path = '../ETL/data/survey_results_20250128_180837.csv'
    
    # Run the analysis
    time_df = analyze_time_differences(csv_path)
    
    # Optionally save to CSV
    output_path = 'time_analysis_results.csv'
    time_df.to_csv(output_path, index=False)
    logger.info(f"Saved time analysis results to {output_path}")

2025-01-28 18:27:21,178 - INFO - Successfully read CSV file with 311 rows
2025-01-28 18:27:21,196 - INFO - Successfully transformed data
2025-01-28 18:27:21,216 - INFO - Saved time analysis results to time_analysis_results.csv



Complete Time Analysis Results:
                             email  video_id  form_startTime  form_endTime   duration_ms  duration_seconds
0          jonahmulcrone@gmail.com  video219    1.737844e+12  1.737844e+12  1.575500e+04      1.575500e+01
1         lisamulcrone02@gmail.com  video537    1.737845e+12  1.737845e+12  1.500700e+04      1.500700e+01
2         lisamulcrone02@gmail.com  video452    1.737845e+12  1.737845e+12  1.525400e+04      1.525400e+01
3         lisamulcrone02@gmail.com   video71    1.737845e+12  1.737845e+12  1.603500e+04      1.603500e+01
4         lisamulcrone02@gmail.com  video188    1.737845e+12  1.737845e+12  1.703600e+04      1.703600e+01
5         lisamulcrone02@gmail.com  video567    1.737845e+12  1.737845e+12  2.065900e+04      2.065900e+01
6         lisamulcrone02@gmail.com   video34    1.737845e+12  1.737845e+12  2.127800e+04      2.127800e+01
7         lisamulcrone02@gmail.com  video249    1.737845e+12  1.737845e+12  2.038900e+04      2.038900e+01
8   

In [83]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import logging
import json

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def clean_gaze_string(gaze_str):
    """Clean and prepare gaze string for JSON parsing."""
    # Remove ObjectId wrapper
    gaze_str = re.sub(r"ObjectId\('([^']+)'\)", r"'\1'", gaze_str)
    
    # Replace single quotes with double quotes
    gaze_str = gaze_str.replace("'", '"')
    
    return gaze_str

def process_gaze_data(csv_path, output_path):
    # Read the CSV file
    df = pd.read_csv(csv_path)
    logger.info(f"Successfully read CSV file with {len(df)} rows")
    
    # Initialize lists to store the processed data
    all_data = []
    max_samples = 0
    
    # Process each row
    for idx, row in df.iterrows():
        try:
            # Get basic info
            base_data = {
                'user_id': row['userId'],
                'video_id': row['videoId'],
                '_id': row['_id']
            }
            
            # Clean and parse the gaze data
            gaze_str = clean_gaze_string(row['gaze'])
            try:
                gaze_data = json.loads(gaze_str)
            except json.JSONDecodeError:
                # If JSON parsing fails, try evaluating as Python literal
                try:
                    import ast
                    gaze_data = ast.literal_eval(row['gaze'])
                except:
                    logger.error(f"Failed to parse gaze data for row {idx}")
                    continue
            
            # Update max samples if this entry has more
            max_samples = max(max_samples, len(gaze_data))
            
            # Add each gaze sample to the base data
            for i, gaze in enumerate(gaze_data):
                try:
                    base_data[f'gaze_{i}_x'] = float(gaze.get('x', np.nan))
                    base_data[f'gaze_{i}_y'] = float(gaze.get('y', np.nan))
                    base_data[f'gaze_{i}_time'] = float(gaze.get('time', np.nan))
                except (ValueError, AttributeError) as e:
                    logger.error(f"Error processing gaze sample {i} in row {idx}: {str(e)}")
                    continue
            
            all_data.append(base_data)
            
            if idx % 10 == 0:
                logger.info(f"Processed {idx} rows...")
            
        except Exception as e:
            logger.error(f"Error processing row {idx}: {str(e)}")
            continue

    # Create DataFrame from processed data
    result_df = pd.DataFrame(all_data)
    
    # Fill NaN values for entries with fewer samples
    result_df = result_df.fillna(np.nan)
    
    # Save to CSV
    result_df.to_csv(output_path, index=False)
    
    logger.info(f"Processing complete. Found {max_samples} maximum gaze samples.")
    logger.info(f"Output saved to {output_path}")
    logger.info(f"Total columns in output: {len(result_df.columns)}")
    
    # Print sample of data
    print("\nColumn names:")
    print(result_df.columns.tolist())
    print("\nFirst few rows of processed data:")
    print(result_df.head())
    
    return result_df

# File paths
input_csv = '../ETL/data/survey_results_20250128_180837.csv'
output_csv = 'gaze_samples_wide_format.csv'

# Process the data
try:
    processed_df = process_gaze_data(input_csv, output_csv)
except Exception as e:
    logger.error(f"Error processing data: {str(e)}")

2025-01-28 18:34:59,235 - INFO - Successfully read CSV file with 311 rows
2025-01-28 18:34:59,236 - INFO - Processed 0 rows...
2025-01-28 18:34:59,237 - INFO - Processed 10 rows...
2025-01-28 18:34:59,238 - INFO - Processed 20 rows...
2025-01-28 18:34:59,239 - INFO - Processed 30 rows...
2025-01-28 18:34:59,241 - INFO - Processed 40 rows...
2025-01-28 18:34:59,242 - INFO - Processed 50 rows...
2025-01-28 18:34:59,243 - INFO - Processed 60 rows...
2025-01-28 18:34:59,245 - INFO - Processed 70 rows...
2025-01-28 18:34:59,247 - INFO - Processed 80 rows...
2025-01-28 18:34:59,248 - INFO - Processed 90 rows...
2025-01-28 18:34:59,250 - INFO - Processed 100 rows...
2025-01-28 18:34:59,251 - INFO - Processed 110 rows...
2025-01-28 18:34:59,253 - INFO - Processed 120 rows...
2025-01-28 18:34:59,255 - INFO - Processed 130 rows...
2025-01-28 18:34:59,258 - INFO - Processed 140 rows...
2025-01-28 18:34:59,260 - INFO - Processed 150 rows...
2025-01-28 18:34:59,262 - INFO - Processed 160 rows...
20


Column names:
['user_id', 'video_id', '_id', 'gaze_0_x', 'gaze_0_y', 'gaze_0_time', 'gaze_1_x', 'gaze_1_y', 'gaze_1_time', 'gaze_2_x', 'gaze_2_y', 'gaze_2_time', 'gaze_3_x', 'gaze_3_y', 'gaze_3_time', 'gaze_4_x', 'gaze_4_y', 'gaze_4_time', 'gaze_5_x', 'gaze_5_y', 'gaze_5_time', 'gaze_6_x', 'gaze_6_y', 'gaze_6_time', 'gaze_7_x', 'gaze_7_y', 'gaze_7_time', 'gaze_8_x', 'gaze_8_y', 'gaze_8_time', 'gaze_9_x', 'gaze_9_y', 'gaze_9_time', 'gaze_10_x', 'gaze_10_y', 'gaze_10_time', 'gaze_11_x', 'gaze_11_y', 'gaze_11_time', 'gaze_12_x', 'gaze_12_y', 'gaze_12_time', 'gaze_13_x', 'gaze_13_y', 'gaze_13_time', 'gaze_14_x', 'gaze_14_y', 'gaze_14_time', 'gaze_15_x', 'gaze_15_y', 'gaze_15_time', 'gaze_16_x', 'gaze_16_y', 'gaze_16_time', 'gaze_17_x', 'gaze_17_y', 'gaze_17_time', 'gaze_18_x', 'gaze_18_y', 'gaze_18_time', 'gaze_19_x', 'gaze_19_y', 'gaze_19_time', 'gaze_20_x', 'gaze_20_y', 'gaze_20_time', 'gaze_21_x', 'gaze_21_y', 'gaze_21_time', 'gaze_22_x', 'gaze_22_y', 'gaze_22_time', 'gaze_23_x', 'gaze

In [85]:
import pandas as pd
import numpy as np

def analyze_max_gaze_samples(csv_path):
    # Read the CSV file
    df = pd.read_csv(csv_path)
    print(f"Total rows in dataset: {len(df)}")
    
    # Get all column names that contain 'gaze' and end with 'x'
    gaze_cols = [col for col in df.columns if 'gaze_' in col and col.endswith('_x')]
    
    # Find the highest gaze sample number
    max_sample = max([int(col.split('_')[1]) for col in gaze_cols])
    print(f"\nHighest gaze sample number: {max_sample}")
    
    # Find rows that have this maximum sample
    max_sample_col = f'gaze_{max_sample}_x'
    rows_with_max = df[df[max_sample_col].notna()]
    print(f"\nNumber of rows with maximum samples: {len(rows_with_max)}")
    
    if len(rows_with_max) > 0:
        # For each row with max samples, calculate the duration
        for idx, row in rows_with_max.iterrows():
            start_time = row['gaze_0_time']
            end_time = row[f'gaze_{max_sample}_time']
            duration = end_time - start_time
            
            print(f"\nRow analysis:")
            print(f"User ID: {row['user_id']}")
            print(f"Video ID: {row['video_id']}")
            print(f"Start Time: {start_time}")
            print(f"End Time: {end_time}")
            print(f"Duration: {duration} milliseconds")
            print(f"Duration: {duration/1000:.2f} seconds")
            
            # Print the first and last gaze samples
            print(f"\nFirst gaze sample:")
            print(f"X: {row['gaze_0_x']}")
            print(f"Y: {row['gaze_0_y']}")
            print(f"Time: {row['gaze_0_time']}")
            
            print(f"\nLast gaze sample:")
            print(f"X: {row[f'gaze_{max_sample}_x']}")
            print(f"Y: {row[f'gaze_{max_sample}_y']}")
            print(f"Time: {row[f'gaze_{max_sample}_time']}")
    
    return rows_with_max

# Analyze the file
gaze_samples_file = 'gaze_samples_wide_format.csv'
max_sample_rows = analyze_max_gaze_samples(gaze_samples_file)

Total rows in dataset: 311

Highest gaze sample number: 673

Number of rows with maximum samples: 1

Row analysis:
User ID: cph82@miami.edu
Video ID: video46
Start Time: 1738038927794.0
End Time: 1738038942602.0
Duration: 14808.0 milliseconds
Duration: 14.81 seconds

First gaze sample:
X: 500.04394239684257
Y: 499.8987181341069
Time: 1738038927794.0

Last gaze sample:
X: 1089.1883695789015
Y: 343.38170799643484
Time: 1738038942602.0
